<h1><center>Barcelona's Transit Accidents 2017-2018<b> Part 1</b>: Raw Dataset Preprocessing</center></h1>

<p style = "font-family:arial;font-size:19px;">This is a dataset of car accidents that occurred during the years of 2017-2018 in the City of Barcelona, Spain. The raw dataset was downloaded from a government page: <a href="https://opendata-ajuntament.barcelona.cat/es/">Open Barcelona</a>.</p> 

<p style = "font-family:arial;font-size:19px;">As a raw dataset, it presents a variety of issues: It is written in Catalan, it has inconsistent labelling, irrelevant information, etc. In this part of the work we will fix these issues, translate the sheets to English, add new columns with categorical or numerical values and left the dataset ready for the exploratory analysis.</p>

<p style = "font-family:arial;font-size:19px;">The first step will be to set our environment with all the required modules that we are going to use in this part of the work.</p>

In [1]:
#Setting Numpy for arrays and calculations:
import numpy as np

#Setting Pandas for Dataframes managing:
import pandas as pd
pd.set_option("display.precision", 4)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

#Setting pyproj for coordinates projections:
from pyproj import Proj

#Not bother with warning messages
import warnings
warnings.filterwarnings('ignore')

<p style = "font-family:arial;font-size:19px;">We have two different Datasheets, one from 2017 and another from 2018. We are going to load both and concatenate them to one named df0.</p>

In [2]:
#Loading the Dataframes:
df2017 = pd.read_csv("2017_accidents_gu_bcn_m.csv", ',')
df2018 = pd.read_csv("2018_accidents_gu_bcn_m.csv", ',')

#Concatenating the Dataframes into one:
df0 = pd.concat([df2017, df2018])

<p style = "font-family:arial;font-size:19px;">We are going to start working with the columns:</p>

In [3]:
#Checking the columns:
df0.columns

Index(['Numero_expedient', 'Codi_districte', 'Nom_districte', 'Codi_barri', 'Nom_barri', 'Codi_carrer', 'Nom_carrer', 'Num_postal', 'Descripcio_dia_setmana', 'Dia_setmana', 'Descripcio_tipus_dia', 'Any', 'Mes_any', 'Nom_mes', 'Dia_mes', 'Hora_dia', 'Descripcio_torn', 'Descripcio_causa_vianant', 'Numero_morts', 'Numero_lesionats_lleus', 'Numero_lesionats_greus', 'Numero_victimes', 'Numero_vehicles_implicats', 'Coordenada_UTM_X', 'Coordenada_UTM_Y'], dtype='object')

<p style = "font-family:arial;font-size:19px;">As we can see, the names of the columns are written in Catalan so we are going to translate them to English:</p>

In [4]:
#We need to map the names of the columns to English:
Catalan = df0.columns.tolist()

#Writting the Names of the Columns in English:
English = ['Id', 'District Name', 'Neighborhood Name', 'Street', 'Weekday', 'Month', 'Day', 'Hour', 'Part of the day', 'Mild injuries', 'Serious injuries', 'Victims', 'Vehicles involved', 'Longitude', 'Latitude']

#Creating the Map:
map_ctoe = {'Numero_expedient': 'Id', 'Codi_districte': 'District number', 'Nom_districte' : 'District Name', 'Codi_barri': 'Neighborhood Number', 'Nom_barri' : 'Neighborhood Name', 'Codi_carrer': 'Street Code', 'Nom_carrer': 'Street', 'Num_postal': "Postal Number", 'Descripcio_dia_setmana': 'Weekday', 'Dia_setmana': "Weekday 2", 'Descripcio_tipus_dia':'Day Description', 'Any': 'Year', 'Mes_any':'Month Number', 'Nom_mes': 'Month', 'Dia_mes' : 'Day', 'Hora_dia': 'Hour', 'Descripcio_torn': 'Part of the Day', 'Descripcio_causa_vianant': 'Cause', 'Numero_morts':'Deaths', 'Numero_lesionats_lleus': 'Mild Injured', 'Numero_lesionats_greus': 'Seriously Injured', 'Numero_victimes' : 'Victims', 'Numero_vehicles_implicats': 'Vehicles Involved', 'Coordenada_UTM_X' : 'UTM_X' , 'Coordenada_UTM_Y' : 'UTM_Y'}

#Mapping:
df0.rename(columns=map_ctoe, inplace=True)

#Checking how the DataFrame looks like now:
df0.head(3)

,Id,District number,District Name,Neighborhood Number,Neighborhood Name,Street Code,Street,Postal Number,Weekday,Weekday 2,Day Description,Year,Month Number,Month,Day,Hour,Part of the Day,Cause,Deaths,Mild Injured,Seriously Injured,Victims,Vehicles Involved,UTM_X,UTM_Y
0,2017S008429,-1,Desconegut,-1,Desconegut,-1,Número 27 ...,Desconegut,Divendres,Dv,Laboral,2017,10,Octubre,13,8,Matí,Altres,0,2,0,2,2,426936.65,4577079.15
1,2017S007316,-1,Desconegut,-1,Desconegut,-1,Número 3 Zona Franca / Número 50 Zona Franca ...,Desconegut,Divendres,Dv,Laboral,2017,9,Setembre,1,13,Matí,No és causa del vianant,0,2,0,2,2,426503.22,4577014.83
2,2017S010210,-1,Desconegut,-1,Desconegut,-1,Litoral (Besòs) ...,K128,Divendres,Dv,Laboral,2017,12,Desembre,8,21,Tarda,No és causa del vianant,0,5,0,5,2,430450.5,4579358.49


<p style = "font-family:arial;font-size:19px;">We can observe that the spatial location of the accidents was written with UTM coordinates. For personal preference and compatibility, it is better to have them in LAT: LONG format. To make this possible we are going to use a specific projection module named "pyproj". The rest of the information needed for the convertion can be obtained from the internet: 
<i style = "font-family:arial;font-size:19px;"><b> Barcelona is in the UTM zone 31T in the North Hemisphere and the UTM coordinates were wrote using datum WGS84.</b></i></p>


In [5]:
#Translating UTM to string, replacing the "," for a '.' and then transforming them to a float64:
UTM_X, UTM_Y = df0['UTM_X'].tolist(), df0['UTM_Y'].tolist()
UTM_X, UTM_Y = [str(i).replace(',','.') for i in UTM_X], [str(i).replace(',','.') for i in UTM_Y]
df0['UTM_X'], df0['UTM_Y'] = [float(i) for i in UTM_X], [float(i) for i in UTM_Y]

#Using pyproj.Proj for the projection:

maplatlong = Proj("+proj=utm +zone=31T, +north +west +ellps=WGS84 +datum=WGS84 +units=m +no_defs")
lon, lat = maplatlong(df0['UTM_X'].tolist(), df0['UTM_Y'].tolist(), inverse=True)

#Making the new columns:
df0['Latitude'] = lat
df0['Longitude'] = lon 

<p style = "font-family:arial;font-size:19px;">We can also observe the presence of some columns that we will not need:</p>
<ul>
    <li style = "font-family:arial;font-size:19px;"><i><b>Day Description</b></i>: Is set to "Laboral" in all the rows.</li>
    <li style = "font-family:arial;font-size:19px;"><i><b>Weekday 2</b></i>: It has the same info that Weekday.</li>
    <li style = "font-family:arial;font-size:19px;"><i><b>UTM_X</b></i> and <i><b>UTM_Y</b></i>: We are going to use the LAT and  LON coordinates.</li>
        <li style = "font-family:arial;font-size:19px;"><i><b>Postal Number</b></i> and <i><b>Street Code</b></i>: They are not consistent and they will not be used in the analysis.</li>
</ul> 


In [6]:
#Checking 'Day Description' column:
print(df0['Day Description'].value_counts())

Laboral    20275
Name: Day Description, dtype: int64


In [7]:
#Day Description is the same in every row so i am going to drop that column:
df0.drop('Day Description', axis=1, inplace=True)

#We don't need 2 columns for the weekday so we can drop the abreviated one:
df0.drop('Weekday 2', axis=1, inplace=True)

#We are going to work with LAT:LONG and not with UTM so we can drop that columns to
df0.drop(['UTM_X','UTM_Y'], axis=1, inplace=True)

#We are not going to use the postal number:
df0.drop('Postal Number', axis=1, inplace=True)

#The Street Code column is not consistent and it is not clear what it means:
df0.drop('Street Code', axis=1, inplace=True)

<p style = "font-family:arial;font-size:19px;">The entries of the categorical columns are written in Catalan so we need to translate them to English too:</p>

In [8]:
print(df0['Weekday'].unique())
print(df0['Month'].unique())
print(df0['Part of the Day'].unique())
print(df0['Cause'].unique())

['Divendres' 'Dijous' 'Dimecres' 'Dissabte' 'Dimarts' 'Dilluns' 'Diumenge']
['Octubre' 'Setembre' 'Desembre' 'Juliol' 'Maig' 'Juny' 'Gener' 'Abril'
 'Març' 'Novembre' 'Febrer' 'Agost']
['Matí' 'Tarda' 'Nit']
['Altres' 'No és causa del  vianant' 'Desobeir el senyal del semàfor'
 'Transitar a peu per la calçada' 'Creuar per fora pas de vianants'
 'Desobeir altres senyals']


In [9]:
#Catalan:
weekdays_c = ['Dilluns', 'Dimarts', 'Dimecres', 'Dijous', 'Divendres', 'Dissabte', 'Diumenge']
months_c = ['Gener', 'Febrer', 'Març', 'Abril', 'Maig', 'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']
parts_of_the_day_c = ['Matí', 'Tarda', 'Nit']
cause_c = ['Altres', 'No és causa del  vianant', 'Desobeir el senyal del semàfor', 'Transitar a peu per la calçada', 'Creuar per fora pas de vianants', 'Desobeir altres senyals']

#English:
weekdays = ['Monday', 'Thursday', 'Wednesday','Tuesday', 'Friday', 'Saturday', 'Sunday']
months = ['January', 'February', 'March', 'April', 'May', 'June', 'July',
          'August', 'September', 'October', 'November', 'December']
parts_of_the_day = ['Morning', 'Afternoon', 'Night']
cause = ['Other', 'Not pedestrian at fault', 'Disobeying the stop light', 'Walking by foot on the road', 'Not crossing in the zebra lines', 'Disobeying other signals']

#Creating the maps:
weekdays_to_english = dict(zip(weekdays_c, weekdays))
months_to_english = dict(zip(months_c, months))
potd_to_english = dict(zip(parts_of_the_day_c, parts_of_the_day))
causes_to_english = dict(zip(cause_c, cause))

#Mapping the values:
df0["Weekday"].replace(weekdays_to_english, inplace=True)
df0["Month"].replace(months_to_english, inplace=True)
df0["Part of the Day"].replace(potd_to_english, inplace=True)
df0["Cause"].replace(causes_to_english, inplace=True)

<p style = "font-family:arial;font-size:19px;">This is how the Dataframe looks like after the translation:</p>

In [10]:
df0.head()

,Id,District number,District Name,Neighborhood Number,Neighborhood Name,Street,Weekday,Year,Month Number,Month,Day,Hour,Part of the Day,Cause,Deaths,Mild Injured,Seriously Injured,Victims,Vehicles Involved,Latitude,Longitude
0,2017S008429,-1,Desconegut,-1,Desconegut,Número 27 ...,Friday,2017,10,October,13,8,Morning,Other,0,2,0,2,2,41.342,2.127
1,2017S007316,-1,Desconegut,-1,Desconegut,Número 3 Zona Franca / Número 50 Zona Franca ...,Friday,2017,9,September,1,13,Morning,Not pedestrian at fault,0,2,0,2,2,41.341,2.122
2,2017S010210,-1,Desconegut,-1,Desconegut,Litoral (Besòs) ...,Friday,2017,12,December,8,21,Afternoon,Not pedestrian at fault,0,5,0,5,2,41.363,2.168
3,2017S006364,-1,Desconegut,-1,Desconegut,Número 3 Zona Franca ...,Friday,2017,7,July,21,2,Night,Not pedestrian at fault,0,1,0,1,2,41.340,2.126
4,2017S004615,10,Sant Martí,64,el Camp de l'Arpa del Clot,Las Navas de Tolosa ...,Tuesday,2017,5,May,25,14,Afternoon,Not pedestrian at fault,0,1,0,1,3,41.418,2.186


<p style = "font-family:arial;font-size:19px;">It is noticeable that the are rows like the first 4 that were added to the Dataframe later in the making process. They have not consistent Neighborhood and District information. We will delete them all.
We also want to check for duplicates entries and delete them.</p>

In [11]:
#Counting the original number of rows:
n_of_rows = df0.shape[0]

#Counting the number of rows with 'Unknown' values:
n_of_p_row = len(df0[(df0['District Name'] == 'Desconegut') | (df0['Neighborhood Name'] == 'Desconegut') ]['Id'].tolist())

#Dropping the rows wich have a couple of columns set to Unknow(Desconegut):
df0 = df0[(df0['District Name'] != 'Desconegut') & (df0['Neighborhood Name'] != 'Desconegut') ].reset_index(drop=True)

#Counting the number of Duplicate entries:
n_of_duplicates = len(df0[df0.duplicated('Id', keep = 'last') == True]['Id'].tolist())

#In addition we want to drop duplicated rows if there are:
df0.drop_duplicates(subset = 'Id', inplace=True)
df0.reset_index(drop=True)

#Counting the final number of columns:
n_of_columns_f = df0.shape[0]

#Printing the changes:
print('There are {} rows in the original datasheet. {} are duplicates entries and {} have "Unknown value".'.format(n_of_rows, n_of_duplicates, n_of_p_row))
print('The final number of rows is {}, {} rows were deleted.'.format(n_of_columns_f, n_of_rows - n_of_columns_f ))

There are 20275 rows in the original datasheet. 14 are duplicates entries and 27 have "Unknown value".
The final number of rows is 20234, 41 rows were deleted.


<p style = "font-family:arial;font-size:19px;">We can see below that the type of each column is right and there is only one row with missing value in the <b>"Street"</b> column. We need to drop this row.

In [12]:
#Checking for rows that we don't want or have some missing values:
df0.info()

#Dropping the missing value row:
df0.dropna(inplace=True);

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20234 entries, 0 to 20247
Data columns (total 21 columns):
Id                     20234 non-null object
District number        20234 non-null int64
District Name          20234 non-null object
Neighborhood Number    20234 non-null int64
Neighborhood Name      20234 non-null object
Street                 20233 non-null object
Weekday                20234 non-null object
Year                   20234 non-null int64
Month Number           20234 non-null int64
Month                  20234 non-null object
Day                    20234 non-null int64
Hour                   20234 non-null int64
Part of the Day        20234 non-null object
Cause                  20234 non-null object
Deaths                 20234 non-null int64
Mild Injured           20234 non-null int64
Seriously Injured      20234 non-null int64
Victims                20234 non-null int64
Vehicles Involved      20234 non-null int64
Latitude               20234 non-null float64
L

<p style = "font-family:arial;font-size:19px;">Regarding the <b>"Street"</b> column we can observe a lot of inconsistency in the way the entries were written:</p>
    <ul>
        <li style = "font-family:arial;font-size:19px;">There are some names written in caps and some written in lower case for the same street.</li>
        <li style = "font-family:arial;font-size:19px;">There are many empty spaces distributed along the entries for the same streets or corners.</li>
        <li style = "font-family:arial;font-size:19px;">Some streets have the "Av" and "Pg" prefixes written on them in only some entries.</li>
        <li style = "font-family:arial;font-size:19px;">The naming scheme of the corners is not consistent neither: The same corners are written "Street A / Street B" for some entries and "Street B / Street A" for others.</li>
    </ul> 

In [13]:
#Checking the document we see that we have problems with the street number and values.
Street_names = df0['Street'].unique().tolist()

#Saving the original number of rows.
print('We have a total of {} different entries for the "Street" column in the original Datasheet'.format(len(Street_names)))
Street_names[0:15]

We have a total of 5862 different entries for the "Street" column in the original Datasheet


['Las Navas de Tolosa                               ',
 'Indústria / Trinxant                              ',
 'Trinxant / Indústria                              ',
 'Indústria                                         ',
 'Maragall                                          ',
 'ST ANTONI M CLARET / Pg Maragall                  ',
 'Sant Antoni Maria Claret / Guinardó               ',
 'Sant Antoni Maria Claret                          ',
 'Còrsega / Còrsega                                 ',
 'CÒRSEGA / Dos de Maig                             ',
 'Rosselló / Independència                          ',
 'VALÈNCIA                                          ',
 'València                                          ',
 'València / Independència                          ',
 'ARAGÓ / Corunya                                   ']

<p style = "font-family:arial;font-size:19px;">In this step we are going to use string methods to remove all the extra white spaces, put all the names in caps and remove the "Av" and "Pg" prefixes.</p>

In [14]:
#We have streets names in different formats so we need to correct this somehow:
df0['Street'].astype(str)
df0['Street']=[(lambda x: x.upper())(x) for x in df0['Street']]

#Getting rid of the prefixes and the blank spaces:
df0['Street']=[(lambda x: x.replace('AV',''))(x) for x in df0['Street']]
df0['Street']=[(lambda x: x.replace('PG',''))(x) for x in df0['Street']]
df0['Street']=[(lambda x: x.replace('\'','`'))(x) for x in df0['Street']]
df0['Street']=[(lambda x: x.strip())(x) for x in df0['Street']]

<p style = "font-family:arial;font-size:19px;">To solve the irregular naming scheme for corners we will use the alphabetical order criterion, for example: A Street / B Street. We are going to remove the extra blank spaces of the names but keep the separation character (" / ").</p>

In [15]:
#Making a function for simetrization:
def simetrization(string, sep,n_split=2):
    newlist = string.split(sep, n_split-1)
    newlist = list(map(str.strip,newlist))
    newlist.sort()
    newlist = sep.join(newlist) 
    return newlist

In [16]:
#Using the function to make symmetric the corners names:
df0['Street'] =[simetrization(x, " / ") for x in df0['Street']]

#Counting the final number of different Street Names:
Street_names2 = df0['Street'].sort_values().unique().tolist()

print('We had {} different Street and Corner Names in the original Datasheet. Now, the final number of different entries is {}.'.format(len(Street_names),len(Street_names2)))
print('{} names were duplicates of the same Streets and Corners'.format(len(Street_names) - len(Street_names2)))

We had 5862 different Street and Corner Names in the original Datasheet. Now, the final number of different entries is 4164.
1698 names were duplicates of the same Streets and Corners


<h><font size="+1" style="font-family:verdana;">In this last step, we are going to add some new categorical columns based in classification criterions. Also, we are going to map some categorical values to numerical values to use them in future calculus.</font></h>

In [17]:
#Adding a Column to account for accidents with more than 1 victim:
df0['Mult_Vic'] = [(lambda x: 'Yes' if x>1 else 'No' )(x) for x in df0['Victims']]

#Adding a Column to account for accidents with more than 1 vehicle:
df0['Mult_Cars'] = [(lambda x: 'Yes' if x>1 else 'No' )(x) for x in df0['Vehicles Involved']]

#Adding a Column to account for accidents made in a corner:
df0['Corner'] = [(lambda x: 'Yes' if '/' in str(x) else 'No' )(x) for x in df0['Street']]

#Adding a Column to account for accidents with serious injuries or Deaths:
df0['Serious'] = [(lambda x: 'Yes' if x>0 else 'No' )(x) for x in df0['Seriously Injured']]
death_list = df0[df0['Deaths'] > 0].index.tolist()
df0.loc[death_list,'Serious'] = 'Yes'


#Mapping days to numerical values and the other way:
dtn = dict(zip(weekdays, list(range(1,8))))
ntd = dict(zip(list(range(1,8)), weekdays))

#Mapping Causes to numerical values and the other way:
cause_tn = dict(zip(cause, list(range(1,7))))
cause_tl = dict(zip(cause, list(range(1,7))))

#Adding a column with the day of the week number to the DataFrame:
df0.insert(loc=df0.columns.get_loc("Weekday") + 1, column='Weekday Number', value=df0.Weekday.replace(dtn))

#Adding a column with the cause number to the DataFrame:
df0.insert(loc=df0.columns.get_loc("Cause") + 1, column='Cause Number', value=df0.Cause.replace(cause_tn))
df0.head(3)

,Id,District number,District Name,Neighborhood Number,Neighborhood Name,Street,Weekday,Weekday Number,Year,Month Number,Month,Day,Hour,Part of the Day,Cause,Cause Number,Deaths,Mild Injured,Seriously Injured,Victims,Vehicles Involved,Latitude,Longitude,Mult_Vic,Mult_Cars,Corner,Serious
0,2017S004615,10,Sant Martí,64,el Camp de l'Arpa del Clot,LAS NAS DE TOLOSA,Tuesday,4,2017,5,May,25,14,Afternoon,Not pedestrian at fault,2,0,1,0,1,3,41.418,2.186,No,Yes,No,No
1,2017S007775,10,Sant Martí,64,el Camp de l'Arpa del Clot,INDÚSTRIA / TRINXANT,Wednesday,3,2017,9,September,20,12,Morning,Not pedestrian at fault,2,0,1,0,1,2,41.418,2.184,No,Yes,Yes,No
2,2017S004484,10,Sant Martí,64,el Camp de l'Arpa del Clot,INDÚSTRIA / TRINXANT,Saturday,6,2017,5,May,20,21,Afternoon,Not pedestrian at fault,2,0,1,0,1,2,41.418,2.184,No,Yes,Yes,No


<p style = "font-family:arial;font-size:19px;">The last step consist in exporting the processed Dataframe to a CSV file that we are going to use in the visual and exploratory analysis.</p>

In [18]:
df0.to_csv('2017_2018_Barcelonas_Accidents.csv', index=False)

<font size="+2">
  <address>
Written by <a href="mailto:NahuelMS@hotmail.com">Nahuel Mussini</a><br>
  </address>
</font>